In [1]:
import pandas as pd
import numpy as np
import keras
import glob
import matplotlib.pyplot as plt
import scipy
from sklearn.metrics import roc_curve, auc
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D, Input, BatchNormalization, Multiply, Activation
from keras.optimizers import RMSprop, SGD
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from keras import backend as K
import os
np.random.seed(2020)

Using TensorFlow backend.


In [6]:
#train files 전처리
train_datagen = ImageDataGenerator(featurewise_center=False,
                 samplewise_center=False,
                 featurewise_std_normalization=False,
                 samplewise_std_normalization=False,
                 rotation_range=20,
                 width_shift_range=0.05,
                 height_shift_range=0.05,
                 shear_range=0.1,
                 zoom_range=0.2,
                 channel_shift_range=0.,
                 fill_mode='nearest',
                 horizontal_flip=True,
                 vertical_flip=False,
                 rescale=1/255,
                 validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
        "food-101\images",
        target_size=(224,224),
        batch_size=64,
        subset='training') # set as training data

#validation set 생성
validation_generator = train_datagen.flow_from_directory(
    "food-101\images",
    target_size=(224,224),
    batch_size=64,
    subset='validation') # set as validation data

Found 24800 images belonging to 31 classes.
Found 6200 images belonging to 31 classes.


In [7]:
#Modeling
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (7,7), strides = 2, padding = 'Same', activation ='relu', input_shape = (224,224,3), kernel_initializer='he_normal'))
model.add(Conv2D(filters = 32, kernel_size = (5,5), strides = 2, padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters = 256, kernel_size = (2,2),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(Conv2D(filters = 512, kernel_size = (2,2),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation = "relu",kernel_initializer='he_normal'))
model.add(Dropout(0.2))
model.add(Dense(31, activation = "softmax",kernel_initializer='he_normal',kernel_regularizer=l2()))


model.compile(optimizer = 'sgd' , loss = "categorical_crossentropy", metrics=["accuracy"])

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 112, 112, 32)      4736      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 56, 56, 32)        25632     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 28, 28, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 28, 28, 64)        18496     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 64)        0         
__________

In [10]:
history = model.fit_generator(train_generator,steps_per_epoch=200,
                              validation_data=validation_generator,validation_steps=750/64, 
                              epochs=100)

Epoch 1/100
200/200 [==============================] - 208s 1s/step - loss: 4.0320 - acc: 0.0359 - val_loss: 3.9979 - val_acc: 0.0495
Epoch 2/100
200/200 [==============================] - 204s 1s/step - loss: 3.9600 - acc: 0.0470 - val_loss: 3.9140 - val_acc: 0.0638
Epoch 3/100
200/200 [==============================] - 202s 1s/step - loss: 3.8817 - acc: 0.0564 - val_loss: 3.8454 - val_acc: 0.0716
Epoch 4/100
200/200 [==============================] - 202s 1s/step - loss: 3.8041 - acc: 0.0679 - val_loss: 3.7739 - val_acc: 0.0859
Epoch 5/100
200/200 [==============================] - 201s 1s/step - loss: 3.7320 - acc: 0.0729 - val_loss: 3.7162 - val_acc: 0.0859
Epoch 6/100
200/200 [==============================] - 202s 1s/step - loss: 3.6822 - acc: 0.0784 - val_loss: 3.6807 - val_acc: 0.0664
Epoch 7/100
200/200 [==============================] - 205s 1s/step - loss: 3.6337 - acc: 0.0892 - val_loss: 3.6106 - val_acc: 0.0872
Epoch 8/100
200/200 [==============================] - 206s 1s

200/200 [==============================] - 206s 1s/step - loss: 2.5878 - acc: 0.2682 - val_loss: 2.5952 - val_acc: 0.2891
Epoch 62/100
200/200 [==============================] - 208s 1s/step - loss: 2.5601 - acc: 0.2809 - val_loss: 2.6080 - val_acc: 0.2773
Epoch 63/100
200/200 [==============================] - 204s 1s/step - loss: 2.5486 - acc: 0.2795 - val_loss: 2.5181 - val_acc: 0.2695
Epoch 64/100
200/200 [==============================] - 204s 1s/step - loss: 2.5635 - acc: 0.2796 - val_loss: 2.6320 - val_acc: 0.2592
Epoch 65/100
200/200 [==============================] - 203s 1s/step - loss: 2.5402 - acc: 0.2905 - val_loss: 2.5799 - val_acc: 0.2826
Epoch 66/100
200/200 [==============================] - 201s 1s/step - loss: 2.5470 - acc: 0.2855 - val_loss: 2.6567 - val_acc: 0.2539
Epoch 67/100
200/200 [==============================] - 204s 1s/step - loss: 2.5182 - acc: 0.2902 - val_loss: 2.5867 - val_acc: 0.2695
Epoch 68/100
200/200 [==============================] - 201s 1s/step

In [11]:
# 모델 평가하기
print("-- Evaluate --")

scores = model.evaluate_generator(
            validation_generator, 
            steps = 5)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
acc: 31.56%


In [12]:
from keras.models import load_model

model.save('food_classfication_git_ver.h5')